In [28]:
import requests
import pandas as pd
import json,os,collections
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time
from tqdm import tqdm

In [29]:
driver = webdriver.Chrome()

In [35]:
def getData(website):
    #website = "https://whop.com/marketplace/boa/"
    driver.get(website)
    wait = WebDriverWait(driver, 10)  # Wait up to 5 seconds
    title_element = wait.until(EC.presence_of_element_located((By.XPATH, "//h1")))
    title = title_element.text
    num_reviews = 0
    try:
        ratings=driver.find_element(By.XPATH,"//div[contains(@class, 'subtitle3 text-whop-black')]").text
        reviews=driver.find_element(By.XPATH,"//div[contains(@class,'text-whop-dark-gray text3 cursor-pointer underline')]").text
        num_reviews = int(reviews.split('(')[-1].split(' ')[0])
        el = driver.find_element('xpath','//div[@class="mb-6 space-y-2"]')
        description = [x.text.strip() for x in el.find_elements('xpath','.//*')]
        description = [x for x in description if x not in ['','Show more']]
        description = description[0].split('\n')
    except:
        ratings="" 
        reviews="" 
        description=""
    Boxes=driver.find_elements(By.XPATH,"//div[contains(@class, 'border-t-whop-stroke')]")
    #Boxes[0].find_elements('xpath','.//div[contains(@class,"font-display")]')[0].text
    els = Boxes[0].find_elements('xpath','.//div[contains(@class,"flex gap-6")]//div[@class="text-header4"]')
    titles= [x.text for x in els]
    els = Boxes[0].find_elements('xpath','.//div[contains(@class,"flex gap-6")]//div[@class="text-paragraph2 text-whop-dark-gray"]')
    parags = [x.text for x in els]
    feature_box_data = {k:v for k,v in zip(titles,parags)}
    els = Boxes[-2].find_elements('xpath','.//p[@class="text-paragraph2 whitespace-nowrap"]')
    about_seller = [x.text for x in els]
    desc = Boxes[-2].find_elements('xpath','.//div[@class="text-paragraph2 text-whop-black whitespace-pre-wrap"]')[0].text
    about_seller.append(desc)
    social_data = []
    socials = Boxes[-2].find_elements('xpath','.//div[@class="border-whop-stroke flex justify-between border-b-[0.5px] py-4"]')
    for social in socials:
        website = social.find_element('xpath','.//div[@class="text-text3"]').text
        website_link = social.find_element('xpath','.//a[@class="text-whop-field-highlight flex cursor-pointer items-center gap-0.5"]').get_attribute('href')
        social_data.append([website,website_link])
    return [title,ratings,num_reviews,description,feature_box_data,about_seller,social_data]

In [31]:
website_list = []
for i in tqdm(range(1,204)):
    url = f"https://whop.com/search/?query=&page={i}"
    driver.get(url)
    time.sleep(5)
    els = driver.find_elements('xpath','//a[@target="_blank"]')
    websites = [x.get_attribute('href') for x in els]
    websites = [x for x in websites if x.find('/marketplace/')!=-1]
    website_list+=websites

100%|██████████| 203/203 [21:23<00:00,  6.33s/it]


In [37]:
website_list=list(set(website_list))
full_webdata = []
for website in tqdm(website_list):
    try:
        web_data  = getData(website)
    except:
        continue
    full_webdata.append(web_data)
df = pd.DataFrame(full_webdata)
df.to_csv('Data.csv',index=False)

  5%|▍         | 94/2019 [04:16<1:27:23,  2.72s/it]


TimeoutException: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF7A5FBAD02+56930]
	(No symbol) [0x00007FF7A5F2F602]
	(No symbol) [0x00007FF7A5DE42E5]
	(No symbol) [0x00007FF7A5E298ED]
	(No symbol) [0x00007FF7A5E29A2C]
	(No symbol) [0x00007FF7A5E6A967]
	(No symbol) [0x00007FF7A5E4BCDF]
	(No symbol) [0x00007FF7A5E681E2]
	(No symbol) [0x00007FF7A5E4BA43]
	(No symbol) [0x00007FF7A5E1D438]
	(No symbol) [0x00007FF7A5E1E4D1]
	GetHandleVerifier [0x00007FF7A6336F8D+3711213]
	GetHandleVerifier [0x00007FF7A63904CD+4077101]
	GetHandleVerifier [0x00007FF7A638865F+4044735]
	GetHandleVerifier [0x00007FF7A6059736+706710]
	(No symbol) [0x00007FF7A5F3B8DF]
	(No symbol) [0x00007FF7A5F36AC4]
	(No symbol) [0x00007FF7A5F36C1C]
	(No symbol) [0x00007FF7A5F268D4]
	BaseThreadInitThunk [0x00007FFC2063257D+29]
	RtlUserThreadStart [0x00007FFC2126AA58+40]


,0,1,2,3,4,5,6
0,Wealth Group,4.93,802,[Wealth Group is currently the top crypto curr...,{'Livestreams': 'Gain access to our exclusive ...,"[802 reviews, Whop Verified, Top Charts, Wealt...","[[Twitter, https://twitter.com/WealthGroup_], ..."
